## YOLO Object Detection

* This is a state of art image detection algorithm known as YOLO(You only look once)

* YOLO can view an image and draw bounding boxes over what it perceives as identified classes.

**What makes YOLO better than other image detection algorithms?**

Prior detection systems repupose classifiers or localizers to perform detection.

They apply the model to an image at multiple locations and scales. High scoring regions of the image are considered detections.

*YOLO uses a totally different approach. We apply a single neural network to the full image. This network divides the image into regions and predicts bounding boxes and probabilities for each region. These bounding boxes are weighted by the predicted probabilities.

*It looks at the whole image at test time so its predictions are informed by global context in the image.

*It also makes predictions with a single network evaluation unlike systems like R-CNN which requires thousands for a single image.

**We will be using a pretrained model that has been trained on COCO dataset. The COCO dataset ha over 1.5 million object instance with 80 different object categories.**

**The reason behind using a pretrined model is that training a model on our own on coco dataset will take hours or days, and very large memory to store data on.**



In [1]:
# Import all the required tools
import time
import os
import cv2
import numpy as np
from model.yolo_model import YOLO # Pre trained model

Using TensorFlow backend.


In [2]:
def process_image(img):
    '''
    Resize, reduce and expand image
    '''
    
    image=cv2.resize(img,(416,416),interpolation=cv2.INTER_CUBIC) # Resizing our image
    
    image=np.array(image,dtype='float32')      # turn into numpy array
    image /=255                                # reduce thr image
    image=np.expand_dims(image,axis=0)         # expand its dimensions
    
    return image

In [3]:
def get_classes(file):
    with open(file) as f:     # Open file
        class_names=f.readlines()    # load its every line 
        
    class_names=[c.strip() for c in class_names]   # put each line in the list as an item
    
    return class_names

In [13]:
def draw(image,boxes,scores,classes,all_classes):
    '''
    Draw the boxes on the image
    '''
    for box,score,cl in zip(boxes,scores,classes):  # zip together boxes,classes and scores
        x,y,w,h=box         # get coordinates
        top=max(0,np.floor(x+0.5).astype(int))     
        left=max(0,np.floor(y+0.5).astype(int))
        right=min(image.shape[1],np.floor(x+w+0.5).astype(int))
        bottom=min(image.shape[0],np.floor(y+h+0.5).astype(int))
        
        cv2.rectangle(image,(top,left),(right,bottom),(255,0,0),2)   # Drawing rectanles around the detected object
        
        cv2.putText(image,'{0} {1:.2f}'.format(all_classes[cl],score),(top,left-6),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,0,255),1,cv2.LINE_AA)
        
        print('class: {0}, score: {1:.2f}'.format(all_classes[cl],score))
        
    print()
        
        
    

In [7]:
def detect_image(image,yolo,all_classes):
    
    pimage=process_image(image)
    start=time.time()
    boxes,classes,scores=yolo.predict(pimage,image.shape)  # Make prediction on the image
    end=time.time()
    
    print('time : {0:.2f}s'.format(end - start))       
    
    if boxes is not None:         
        draw(image,boxes,scores,classes,all_classes)
        
    return image

In [10]:
yolo=YOLO(0.4,0.5)       # Threshold values
file='data/coco_classes.txt'   # our classes file
all_classes=get_classes(file)

### Detecting Images 

In [11]:
f='bundo-kim-syhh8U3R55E-unsplash.jpg'
path='images/test/' + f
image=cv2.imread(path)
image=detect_image(image,yolo,all_classes)
cv2.imwrite('images/res/'+f,image)


time : 17.70s
class: person, score: 0.95
class: dog, score: 0.99
class: dog, score: 0.88
class: car, score: 0.94
class: car, score: 0.89
class: handbag, score: 0.48



True

In [14]:
f='gena-okami-yaHhqZvxjMw-unsplash.jpg'
path='images/test/' + f
image=cv2.imread(path)
image=detect_image(image,yolo,all_classes)
cv2.imwrite('images/res/'+f,image)

time : 14.47s
class: dog, score: 0.94
class: car, score: 0.75
class: car, score: 0.44
class: car, score: 0.43
class: truck, score: 0.88



True